url: https://analysis-navi.com/?p=569

url: https://www.youtube.com/watch?v=5vRyPMBOr_w&t=1s

In [1]:
# !apt-get remove mecab libmecab-dev mecab-ipadic-utf8
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [2]:
# https://github.com/SamuraiT/mecab-python3#common-issues
!pip install unidic-lite

In [3]:
### 形態素解析
import MeCab

# text = "私はお昼休みに美味しいカレーライスを食べました。"

m = MeCab.Tagger()

def mecab_sep(text):
    node = m.parseToNode(text)

    words_list = []

    while node:
        if node.feature.split(",")[0] == "動詞":
            words_list.append(node.feature.split(",")[6])
        elif node.feature.split(",")[0] == "形容詞":
            words_list.append(node.feature.split(",")[6])
        else:
            words_list.append(node.surface)

        node = node.next

    return words_list[1:-1]

In [4]:
text = "私はお昼休みに美味しいカレーライスを食べました。"
text_2 = "私は夜に美味しいラーメンを作りました。"

display(mecab_sep(text))
display(mecab_sep(text_2))

['私', 'は', 'お', '昼休み', 'に', 'オイシイ', 'カレー', 'ライス', 'を', 'タベル', 'まし', 'た', '。']

['私', 'は', '夜', 'に', 'オイシイ', 'ラーメン', 'を', 'ツクル', 'まし', 'た', '。']

In [5]:
### Bag-of-words / tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

def calc_vecs(docs):
    vectorizer = TfidfVectorizer(analyzer=mecab_sep)
    vecs = vectorizer.fit_transform(docs)
    return vecs.toarray()

    TfidfVectorizerのインスタンス生成:
    TfidfVectorizerはテキストデータをTF-IDFベクトルに変換するためのクラスです。
    このクラスのインスタンスを生成し、analyzer引数に単語分割を行う関数（mecab_sep）を指定しています。

    fit_transformメソッドの実行:
    fit_transformメソッドを使用して、テキストデータ（docs）をTF-IDFベクトルに変換します。
    このメソッドは、文書全体のトークン（単語）の統計情報を学習し、TF-IDFベクトルに変換します。
    
    密行列への変換:
    toarrayメソッドを使用して、疎行列形式のTF-IDFベクトルを密行列（NumPy配列）に変換します。

In [6]:
# tf-idfのベクトル(シンプル)
vectorizer_tf_idf = TfidfVectorizer(analyzer=mecab_sep)
vecs_tf_idf = vectorizer_tf_idf.fit_transform([text, text_2])
display(vecs_tf_idf.toarray())

# Bag-of-wordsのベクトル
vectorizer_bag_of_words = TfidfVectorizer(analyzer=mecab_sep, use_idf=False)
vecs_bag_of_words = vectorizer_bag_of_words.fit_transform([text, text_2])
display(vecs_bag_of_words.toarray())

# Bag-of-wordsのベクトル(binary_version)
vectorizer_bag_of_words = TfidfVectorizer(analyzer=mecab_sep, binary=True, use_idf=False)
vecs_bag_of_words = vectorizer_bag_of_words.fit_transform([text, text_2])
display(vecs_bag_of_words.toarray())

array([[0.23651397, 0.33241213, 0.23651397, 0.23651397, 0.23651397,
        0.23651397, 0.23651397, 0.23651397, 0.33241213, 0.33241213,
        0.        , 0.33241213, 0.        , 0.        , 0.33241213,
        0.23651397],
       [0.26797042, 0.        , 0.26797042, 0.26797042, 0.26797042,
        0.26797042, 0.26797042, 0.26797042, 0.        , 0.        ,
        0.37662308, 0.        , 0.37662308, 0.37662308, 0.        ,
        0.26797042]])

array([[0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 ,
        0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 ,
        0.        , 0.2773501 , 0.        , 0.        , 0.2773501 ,
        0.2773501 ],
       [0.30151134, 0.        , 0.30151134, 0.30151134, 0.30151134,
        0.30151134, 0.30151134, 0.30151134, 0.        , 0.        ,
        0.30151134, 0.        , 0.30151134, 0.30151134, 0.        ,
        0.30151134]])

array([[0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 ,
        0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 , 0.2773501 ,
        0.        , 0.2773501 , 0.        , 0.        , 0.2773501 ,
        0.2773501 ],
       [0.30151134, 0.        , 0.30151134, 0.30151134, 0.30151134,
        0.30151134, 0.30151134, 0.30151134, 0.        , 0.        ,
        0.30151134, 0.        , 0.30151134, 0.30151134, 0.        ,
        0.30151134]])

In [7]:
### コサイン類似度
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([[1,2,3]],[[4,5,6], [4,3,2], [1,6,3]])

array([[0.97463185, 0.79406667, 0.86692145]])

In [8]:
### コサイン類似度
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

input_doc = "私はネコが好きです。" #比較対象

target_docs_df = pd.read_csv("/content/nlp_datascience_cram_school_2.csv", encoding='shift_jis')
target_docs = target_docs_df["文章リスト"].tolist()

display(target_docs_df)
display(target_docs)

,文章リスト
0,私は犬が好きです。
1,私は犬が嫌いです。
2,私は犬がとても好きです。
3,私は犬と猫が好きです。
4,私は猫が好きです。


['私は犬が好きです。', '私は犬が嫌いです。', '私は犬がとても好きです。', '私は犬と猫が好きです。', '私は猫が好きです。']

In [9]:
all_docs = [input_doc] + target_docs
display(all_docs)

all_docs_vecs = calc_vecs(all_docs)
display(all_docs_vecs)

similarity = cosine_similarity([all_docs_vecs[0]], all_docs_vecs[1:])
display(similarity)

target_docs_df["類似度"] = similarity[0]
display(target_docs_df)

['私はネコが好きです。',
 '私は犬が好きです。',
 '私は犬が嫌いです。',
 '私は犬がとても好きです。',
 '私は犬と猫が好きです。',
 '私は猫が好きです。']

array([[0.29608349, 0.29608349, 0.29608349, 0.        , 0.        ,
        0.29608349, 0.66700592, 0.34172496, 0.        , 0.        ,
        0.        , 0.29608349],
       [0.35096963, 0.35096963, 0.35096963, 0.        , 0.        ,
        0.35096963, 0.        , 0.40507184, 0.        , 0.46906117,
        0.        , 0.35096963],
       [0.2903605 , 0.2903605 , 0.2903605 , 0.        , 0.        ,
        0.2903605 , 0.        , 0.        , 0.65411338, 0.38805875,
        0.        , 0.2903605 ],
       [0.27531225, 0.27531225, 0.27531225, 0.        , 0.62021325,
        0.27531225, 0.        , 0.31775182, 0.        , 0.36794718,
        0.        , 0.27531225],
       [0.24539842, 0.24539842, 0.24539842, 0.55282448, 0.        ,
        0.24539842, 0.        , 0.28322676, 0.        , 0.32796818,
        0.45332398, 0.24539842],
       [0.32034547, 0.32034547, 0.32034547, 0.        , 0.        ,
        0.32034547, 0.        , 0.36972694, 0.        , 0.        ,
        0.5917735 ,

array([[0.65800473, 0.42985475, 0.51616079, 0.46007776, 0.60058995]])

,文章リスト,類似度
0,私は犬が好きです。,0.658005
1,私は犬が嫌いです。,0.429855
2,私は犬がとても好きです。,0.516161
3,私は犬と猫が好きです。,0.460078
4,私は猫が好きです。,0.600590


In [10]:
target_docs_df.sort_values("類似度",ascending=False)

,文章リスト,類似度
0,私は犬が好きです。,0.658005
4,私は猫が好きです。,0.600590
2,私は犬がとても好きです。,0.516161
3,私は犬と猫が好きです。,0.460078
1,私は犬が嫌いです。,0.429855


In [11]:
import glob
target_docs=[]

for x in glob.glob("text/*/*.txt"):
    text = ""
    with open(x) as f:
        next(f)
        next(f)
        for line in f:
            text = text + line
    f.close()
    target_docs.append(text)


    glob.glob("text/*/*.txt") は、"text"ディレクトリ以下に存在するすべてのサブディレクトリ内のテキストファイルに対してパターンマッチングを行います。

    ファイルが見つかるたびに、それを読み込みます。
    最初の2行をnext(f)で無視し、残りの行をテキストデータに連結します。
    ファイルを閉じます。

    テキストデータを target_docs リストに追加します。

    このスクリプトは、指定されたディレクトリ内のテキストファイルからデータを収集し、そのテキストデータを target_docs というリストに格納します。
    このリストは後続の処理に使用されるでしょう。